# Data Mining - Project
## Data Preparation Notebook
## Group Members
|Name|StudentID|
|:-----------:|:--------:|
|Fabio Lopes|20200597|
|Filipe Costa|20201041|
|Jorge Pereira|20201085|


In [1]:
import sys
!{sys.executable} -m pip install pyshp

You should consider upgrading via the 'C:\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import shapefile as shp

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import MinMaxScaler

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df_source = pd.read_csv('./data/donors.csv')
df_source.drop('Unnamed: 0', axis = 1, inplace = True)

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_source.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,HOMEOWNR,CHILD03,CHILD07,CHILD12,CHILD18,NUMCHLD,INCOME,GENDER,WEALTH1,HIT,MBCRAFT,MBGARDEN,MBBOOKS,MBCOLECT,MAGFAML,MAGFEM,MAGMALE,PUBGARDN,PUBCULIN,PUBHLTH,PUBDOITY,PUBNEWFN,PUBPHOTO,PUBOPP,DATASRCE,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,STATEGOV,FEDGOV,SOLP3,SOLIH,MAJOR,WEALTH2,GEOCODE,COLLECT1,VETERANS,BIBLE,CATLG,HOMEE,PETS,CDPLAY,STEREO,PCOWNERS,PHOTO,CRAFTS,FISHER,GARDENIN,BOATS,WALKER,KIDSTUFF,CARDS,PLATES,LIFESRC,PEPSTRFL,POP901,POP902,POP903,POP90C1,POP90C2,POP90C3,POP90C4,POP90C5,ETH1,ETH2,ETH3,ETH4,ETH5,ETH6,ETH7,ETH8,ETH9,ETH10,ETH11,ETH12,ETH13,ETH14,ETH15,ETH16,AGE901,AGE902,AGE903,AGE904,AGE905,AGE906,AGE907,CHIL1,CHIL2,CHIL3,AGEC1,AGEC2,AGEC3,AGEC4,AGEC5,AGEC6,AGEC7,CHILC1,CHILC2,CHILC3,CHILC4,CHILC5,HHAGE1,HHAGE2,HHAGE3,HHN1,HHN2,HHN3,HHN4,HHN5,HHN6,MARR1,MARR2,MARR3,MARR4,HHP1,HHP2,DW1,DW2,DW3,DW4,DW5,DW6,DW7,DW8,DW9,HV1,HV2,HV3,HV4,HU1,HU2,HU3,HU4,HU5,HHD1,HHD2,HHD3,HHD4,HHD5,HHD6,HHD7,HHD8,HHD9,HHD10,HHD11,HHD12,ETHC1,ETHC2,ETHC3,ETHC4,ETHC5,ETHC6,HVP1,HVP2,HVP3,HVP4,HVP5,HVP6,HUR1,HUR2,RHP1,RHP2,RHP3,RHP4,HUPA1,HUPA2,HUPA3,HUPA4,HUPA5,HUPA6,HUPA7,RP1,RP2,RP3,RP4,MSA,ADI,DMA,IC1,IC2,IC3,IC4,IC5,IC6,IC7,IC8,IC9,IC10,IC11,IC12,IC13,IC14,IC15,IC16,IC17,IC18,IC19,IC20,IC21,IC22,IC23,HHAS1,HHAS2,HHAS3,HHAS4,MC1,MC2,MC3,TPE1,TPE2,TPE3,TPE4,TPE5,TPE6,TPE7,TPE8,TPE9,PEC1,PEC2,TPE10,TPE11,TPE12,TPE13,LFC1,LFC2,LFC3,LFC4,LFC5,LFC6,LFC7,LFC8,LFC9,LFC10,OCC1,OCC2,OCC3,OCC4,OCC5,OCC6,OCC7,OCC8,OCC9,OCC10,OCC11,OCC12,OCC13,EIC1,EIC2,EIC3,EIC4,EIC5,EIC6,EIC7,EIC8,EIC9,EIC10,EIC11,EIC12,EIC13,EIC14,EIC15,EIC16,OEDC1,OEDC2,OEDC3,OEDC4,OEDC5,OEDC6,OEDC7,EC1,EC2,EC3,EC4,EC5,EC6,EC7,EC8,SEC1,SEC2,SEC3,SEC4,SEC5,AFC1,AFC2,AFC3,AFC4,AFC5,AFC6,VC1,VC2,VC3,VC4,ANC1,ANC2,ANC3,ANC4,ANC5,ANC6,ANC7,ANC8,ANC9,ANC10,ANC11,ANC12,ANC13,ANC14,ANC15,POBC1,POBC2,LSC1,LSC2,LSC3,LSC4,VOC1,VOC2,VOC3,HC1,HC2,HC3,HC4,HC5,HC6,HC7,HC8,HC9,HC10,HC11,HC12,HC13,HC14,HC15,HC16,HC17,HC18,HC19,HC20,HC21,MHUC1,MHUC2,AC1,AC2,ADATE_2,ADATE_3,ADATE_4,ADATE_5,ADATE_6,ADATE_7,ADATE_8,ADATE_9,ADATE_10,ADATE_11,ADATE_12,ADATE_13,ADATE_14,ADATE_15,ADATE_16,ADATE_17,ADATE_18,ADATE_19,ADATE_20,ADATE_21,ADATE_22,ADATE_23,ADATE_24,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,RFA_8,RFA_9,RFA_10,RFA_11,RFA_12,RFA_13,RFA_14,RFA_15,RFA_16,RFA_17,RFA_18,RFA_19,RFA_20,RFA_21,RFA_22,RFA_23,RFA_24,CARDPROM,MAXADATE,NUMPROM,CARDPM12,NUMPRM12,RDATE_3,RDATE_4,RDATE_5,RDATE_6,RDATE_7,RDATE_8,RDATE_9,RDATE_10,RDATE_11,RDATE_12,RDATE_13,RDATE_14,RDATE_15,RDATE_16,RDATE_17,RDATE_18,RDATE_19,RDATE_20,RDATE_21,RDATE_22,RDATE_23,RDATE_24,RAMNT_3,RAMNT_4,RAMNT_5,RAMNT_6,RAMNT_7,RAMNT_8,RAMNT_9,RAMNT_10,RAMNT_11,RAMNT_12,RAMNT_13,RAMNT_14,RAMNT_15,RAMNT_16,RAMNT_17,RAMNT_18,RAMNT_19,RAMNT_20,RAMNT_21,RAMNT_22,RAMNT_23,RAMNT_24,RAMNTALL,NGIFTALL,CARDGIFT,MINRAMNT,MINRDATE,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,,,,XXXX,T2,,,,,,NaN,NaN,F,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0,39,34,18,10,2,1,,,,5.0,,,,,,,,,,,,,,,,,,,,,X,992,264,332,0,35,65,47,53,92,1,0,0,11,0,0,0,0,0,0,0,11,0,0,0,39,48,51,40,50,54,25,31,42,27,11,14,18,17,13,11,15,12,11,34,25,18,26,10,23,18,33,49,28,12,4,61,7,12,19,198,276,97,95,2,2,0,0,7,7,0,479,635,3,2,86,14,96,4,7,38,80,70,32,84,16,6,2,5,9,15,3,17,50,25,0,0,0,2,7,13,27,47,0,1,61,58,61,15,4,2,0,0,14,1,0,0,2,5,17,73,0.0,177.0,682.0,307,318,349,378,12883,13,23,23,23,15,1,0,0,1,4,25,24,26,17,2,0,0,2,28,4,51,1,46,54,3,88,8,0,0,0,0,0,0,4,1,13,14,16,2,45,56,64,50,64,44,62,53,99,0,0,9,3,8,13,9,0,3,9,3,15,19,5,4,3,0,3,41,1,0,7,13,6,5,0,4,9,4,1,3,10,2,1,7,78,2,0,120,16,10,39,21,8,4,3,5,20,3,19,4,0,0,0,18,39,0,34,23,18,16,1,4,0,23,0,0,5,1,0,0,0,0,0,2,0,3,74,88,8,0,4,96,77,19,13,31,5,14,14,31,54,46,0,0,90,0,10,0,0,0,33,65,40,99,99,6,2,10,7,2017-06-01,2016-06-01,2016-04-01,2016-04-01,2016-03-01,2016-02-01,20

In [6]:
def splitNumericAndCategorical(df):
    """Informs user on the total number of columns in the dataframe and splits it
       in 2 dataframes, one numerical and the other categorical."""
    df_num = df._get_numeric_data()
    df_cat = df.drop(df_num.columns, axis = 1)
    print(f"Total Columns: {df.shape[1]}")
    print(f"Numerical Columns: {df_num.shape[1]}")
    print(f"Categorical Columns: {df_cat.shape[1]}")
    return df_num,df_cat

In [7]:
df_numeric, df_categorical = splitNumericAndCategorical(df_source)

Total Columns: 475
Numerical Columns: 350
Categorical Columns: 125


# Drop columns which are not relevant for the analysis

In [8]:
df_aux1 = pd.DataFrame()

with open('ColumnsToKeep.txt','r') as fp:
    ColumnsToKeep = [line.strip() for line in fp.readlines()]

for column in ColumnsToKeep:
    df_aux1[column] = df_source[column]

In [9]:
df_numeric, df_categorical = splitNumericAndCategorical(df_aux1)

Total Columns: 341
Numerical Columns: 311
Categorical Columns: 30


# Prepare data

In [10]:
df_aux2 = df_aux1.copy()

## Cast dates to datetime object + create variables

In [11]:
DateColumns = ['DOB','MAXADATE','MINRDATE','MAXRDATE','LASTDATE','FISTDATE','NEXTDATE']

for column in DateColumns:
    df_aux2[column] = pd.to_datetime(df_aux1[column])

In [12]:
# Computing AGE from DOB
df_aux2['AGE'] = (pd.Timestamp('now') - df_aux2['DOB']).astype('<m8[Y]')
df_aux2.drop('DOB', axis = 1, inplace = True)

# Computing number of days elapsed from each date variable
DateColumns = ['MAXADATE','MINRDATE','MAXRDATE','LASTDATE','FISTDATE','NEXTDATE']
for column in DateColumns:
    df_aux2[f'elapsed_{column}'] = (pd.Timestamp('now') - df_aux2[column]).dt.days
    
df_aux2.drop(DateColumns, axis = 1, inplace = True)

## Create Binary Variables based on the flags

In [13]:
df_aux3 = df_aux2.copy()

In [14]:
df_aux3['PVASTATE'] = df_aux2['PVASTATE'].apply(lambda x: 1 if ( (x == 'E') or (x == 'P')) else 0)
df_aux3['NOEXCH'  ] = df_aux2['NOEXCH'  ].apply(lambda x: 0 if ( (x == 'X') ) else 1)
df_aux3['HOMEOWNR'] = df_aux2['HOMEOWNR'].apply(lambda x: 0 if ( (x == 'H') ) else 1)
df_aux3['VETERANS'] = df_aux2['VETERANS'].apply(lambda x: 0 if ( (x == 'Y') ) else 1)
df_aux3['RECINHSE'] = df_aux2['RECINHSE'].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['RECP3'   ] = df_aux2['RECP3'   ].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['RECPGVG' ] = df_aux2['RECPGVG' ].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['RECSWEEP'] = df_aux2['RECSWEEP'].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['MAJOR'   ] = df_aux2['MAJOR'   ].apply(lambda x: 1 if ( (x == 'X') ) else 0)
df_aux3['PEPSTRFL'] = df_aux2['PEPSTRFL'].apply(lambda x: 1 if ( (x == 'X') ) else 0)

## Fixes issues identified in some columns

In [15]:
df_aux4 = df_aux3.copy()

In [16]:
df_aux4['ZIP'] = df_aux3['ZIP'].str.replace('-','').astype(int)
df_aux4['GENDER'] = df_aux3['GENDER'].apply(lambda x: 1 if (x == 'F') else 2 if (x == 'M') else 0)
df_aux4['DATASRCE'] = df_aux3['DATASRCE'].apply(lambda x: 0 if (x == ' ') else int(x))
df_aux4['SOLP3'] = df_aux3['SOLP3'].apply(lambda x: 12 if (x == ' ') else int(x))
df_aux4['SOLIH'] = df_aux3['SOLIH'].apply(lambda x: 12 if (x == ' ') else int(x))
df_aux4['GEOCODE'] = df_aux3['GEOCODE'].apply(lambda x: 0 if (x == ' ') else int(x))

## Handle Domain, RFA, MDAUD Columns

In [17]:
df_aux5 = df_aux4.copy()

### Domain

In [18]:
# DOMAIN handling
# 1st byte = Urbanicity level of the donor's neighborhood
#     U=Urban=5
#     C=City=4
#     S=Suburban=3
#     T=Town=2
#     R=Rural=1

# 2nd byte = Socio-Economic status of the neighborhood
#     1 = Highest SES
#     2 = Average SES
#     3 = Lowest SES 
#     (except for Urban communities, where
#       1 = Highest SES, 
#       2 = Above average SES,
#       3 = Below average SES, 
#       4 = Lowest SES.)

def prepURBANICITY(Code):
    if Code == ' ':
        return 0 #Empty
    else:
        if Code[0] == 'R':
            return 1
        elif Code[0] == 'T':
            return 2
        elif Code[0] == 'S':
            return 3
        elif Code[0] == 'C':
            return 4
        elif Code[0] == 'U':
            return 5
        else:
            return 0
        
def prepSOCIOECON(Code):
    if Code == ' ':
        return 0 #Empty
    else:
        if (Code[0] == 'R') or (Code[0] == 'T') or (Code[0] == 'S') or Code[0] == 'C':
            if Code[1] == '1':
                return 1
            elif Code[1] == '2':
                return 3
            elif Code[1] == '3':
                return 5
            else:
                return 0
        elif Code[0] == 'U':
            if Code[1] == '1':
                return 1
            elif Code[1] == '2':
                return 2
            elif Code[1] == '3':
                return 4
            elif Code[1] == '4':
                return 5
            else:
                return 0
        else:
            return 0

df_aux5['URBANICITY'] = df_aux4['DOMAIN'].apply(prepURBANICITY)
df_aux5['SOCIOECON' ] = df_aux4['DOMAIN'].apply(prepSOCIOECON)
df_aux5.drop('DOMAIN', axis = 1, inplace = True)

### RFA

In [19]:
def prepLASTGIFTAMOUNTCATEGORY(Code):
    if Code == ' ':
        return 0 #Empty
    else:
        if Code[0] == 'A':
            return 1
        elif Code[0] == 'B':
            return 2
        elif Code[0] == 'C':
            return 3
        elif Code[0] == 'D':
            return 4
        elif Code[0] == 'E':
            return 5
        elif Code[0] == 'F':
            return 6
        elif Code[0] == 'G':
            return 7
        else:
            return 0
        
df_aux5['LASTGIFTAMOUNTCATEGORY'] = df_aux4['RFA_2A'].apply(prepLASTGIFTAMOUNTCATEGORY)
df_aux5.drop('RFA_2A', axis = 1, inplace = True)
df_aux5.drop('RFA_2R', axis = 1, inplace = True) # all donors are lapsing donors

### MDAUD

In [20]:
def prepMDONOR_GIVING_RECENCY(Code):
    if Code[0] == 'X':
        return 0 # Not a major donor
    else:
        if Code[0] == 'C':
            return 1
        elif Code[0] == 'L':
            return 2
        elif Code[0] == 'I':
            return 3
        elif Code[0] == 'D':
            return 4
        else:
            return 0
    
def prepMDONOR_GIVING_FREQUENCY(Code):
    if Code[0] == 'X':
        return 0 # Not a major donor
    else:
        if Code[0] == '1':
            return 1
        elif Code[0] == '2':
            return 2
        elif Code[0] == '5':
            return 3
        else:
            return 0
        
def prepMDONOR_GIVING_AMOUNT(Code):
    if Code[0] == 'X':
        return 0 # Not a major donor
    else:
        if Code[0] == 'L':
            return 1
        elif Code[0] == 'C':
            return 2
        elif Code[0] == 'M':
            return 3
        elif Code[0] == 'T':
            return 4
        else:
            return 0
        
df_aux5['MDONOR_GIVING_RECENCY'] = df_aux4['MDMAUD_R'].apply(prepMDONOR_GIVING_RECENCY)
df_aux5['MDONOR_GIVING_FREQUENCY'] = df_aux4['MDMAUD_F'].apply(prepMDONOR_GIVING_FREQUENCY)
df_aux5['MDONOR_GIVING_AMOUNT'] = df_aux4['MDMAUD_A'].apply(prepMDONOR_GIVING_AMOUNT)

df_aux5.drop('MDMAUD_R', axis = 1, inplace = True)
df_aux5.drop('MDMAUD_F', axis = 1, inplace = True)
df_aux5.drop('MDMAUD_A', axis = 1, inplace = True)

In [21]:
df_numeric, df_categorical = splitNumericAndCategorical(df_aux5)

Total Columns: 341
Numerical Columns: 340
Categorical Columns: 1


In [22]:
df_Prepped = df_aux5.copy()

# Scaling the data

In [23]:
df_scaling = df_Prepped.copy()

In [24]:
df_numeric, df_categorical = splitNumericAndCategorical(df_scaling)

Total Columns: 341
Numerical Columns: 340
Categorical Columns: 1


In [25]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaling[df_numeric.columns] = scaler.fit_transform(df_numeric)

# Reduce Dimensionality

In [26]:
# https://www.kdnuggets.com/2015/05/7-methods-data-dimensionality-reduction.html

In [27]:
df_FEreduction = df_scaling.copy()

## Drop columns based on valid observations

In [28]:
df_numeric, df_categorical = splitNumericAndCategorical(df_FEreduction)

Total Columns: 341
Numerical Columns: 340
Categorical Columns: 1


In [29]:
import math

ThresholdToIgnore = 0.4 # If more than 40% of the dataset is missing

MinimumValidSamples = math.floor(df_FEreduction.shape[0] * ThresholdToIgnore)
print(f"Minimum Valid Samples: {MinimumValidSamples}")
for column in df_FEreduction.columns:
    count = df_FEreduction[column].count()
    #print(f"{column}:{count}")
    if count < MinimumValidSamples:
        print(f"Dropped {column} , Valid Observations: {count}")
        df_FEreduction.drop(column,axis=1, inplace=True)

Minimum Valid Samples: 38164
Dropped NUMCHLD , Valid Observations: 12386


## Drop columns with high/low correlation

In [30]:
df_numeric, df_categorical = splitNumericAndCategorical(df_FEreduction)

Total Columns: 340
Numerical Columns: 339
Categorical Columns: 1


In [31]:
corPearson = df_numeric.corr().abs()

# Select upper triangle of correlation matrix
upper = corPearson.where(np.triu(np.ones(corPearson.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.90 and lower than 0.3
HighCorrelation = [column for column in upper.columns if any(upper[column] > 0.90)]
LowCorrelation = [column for column in upper.columns if max(upper[column]) < 0.30]

print(f"Dropping {len(HighCorrelation)} columns with High correlation.")
print(f"Dropping {len(LowCorrelation)} columns with Low correlation.")

# Drop features 
df_FEreduction.drop(HighCorrelation, axis=1, inplace=True)
df_FEreduction.drop(LowCorrelation, axis=1, inplace=True)

Dropping 66 columns with High correlation.
Dropping 55 columns with Low correlation.


## Drop columns with low variance

In [32]:
df_numeric, df_categorical = splitNumericAndCategorical(df_FEreduction)

Total Columns: 219
Numerical Columns: 218
Categorical Columns: 1


In [33]:
VarianceThreshold = 0.005

for column in df_numeric:
    var = df_FEreduction[column].var()
    if VarianceThreshold > var:
        print(f"Dropped {column} , Variance: {var}")
        df_FEreduction.drop(column,axis=1,inplace=True)

Dropped ETH6 , Variance: 0.003813343269872504
Dropped ETH7 , Variance: 0.003844572899357172
Dropped ETH8 , Variance: 0.0026200820042238814
Dropped ETH9 , Variance: 0.004751550371007165
Dropped ETH10 , Variance: 0.001903727204597813
Dropped ETH11 , Variance: 0.00201207974637391
Dropped ETH12 , Variance: 0.0010556568291769696
Dropped HHD8 , Variance: 0.002021027601848559
Dropped ETHC6 , Variance: 0.0034788809233272065
Dropped RHP4 , Variance: 0.003498429401515511
Dropped HUPA7 , Variance: 0.0037265213687236657
Dropped IC13 , Variance: 0.003975778425846497
Dropped TPE5 , Variance: 0.0038331966102909364
Dropped OCC12 , Variance: 0.004239876371784107
Dropped OCC13 , Variance: 0.0036243906469866365
Dropped EIC5 , Variance: 0.004183862269308264
Dropped EIC11 , Variance: 0.004250683752400797
Dropped AFC1 , Variance: 0.004270528734168984
Dropped AFC3 , Variance: 0.0007472538414232655
Dropped ANC7 , Variance: 0.004503301954945094
Dropped ANC12 , Variance: 0.003009620016266497
Dropped AC1 , Varia

## Remove Neighborhood data

In [34]:
with open('NeighborhoodColumns.txt','r') as fp:
    NeighborhoodColumns = [line.strip() for line in fp.readlines()]

NeighborhoodColumnsToKeep = []

columnsToDrop = list(set(NeighborhoodColumns) - set(NeighborhoodColumnsToKeep))
    
for column in columnsToDrop:
    if column in df_FEreduction.columns:
        df_FEreduction.drop(column,axis=1,inplace=True)

## Choose Clustering Variables

In [35]:
dropColumns = ['ZIP','RECSWEEP','DATASRCE','URBANICITY','SOCIOECON']

df_FEreduction.drop(dropColumns,axis=1,inplace=True)

In [46]:
df_FEreduction

,STATE,INCOME,WEALTH1,WWIIVETS,FEDGOV,WEALTH2,CARDPROM,CARDPM12,NUMPRM12,NGIFTALL,RFA_2F,elapsed_MINRDATE,elapsed_MAXRDATE,elapsed_LASTDATE,LASTGIFTAMOUNTCATEGORY
0,IL,NaN,NaN,-0.636364,-0.977011,0.111111,-0.133333,-0.368421,-0.662338,-0.745763,1.000000,-0.584386,-0.718758,0.217639,-0.333333
1,CA,0.666667,1.000000,-0.777778,-0.977011,1.000000,-0.633333,-0.368421,-0.688312,-0.983051,-0.333333,-0.692016,-0.890172,0.217639,1.000000
2,NC,-0.333333,-0.777778,-0.333333,-0.977011,-0.777778,-0.166667,-0.368421,-0.662338,-0.779661,1.000000,-0.515159,-0.569926,0.217639,-0.333333
3,CA,-1.000000,-0.111111,-0.373737,-0.931034,-1.000000,-0.133333,-0.368421,-0.662338,-0.872881,1.000000,-0.146033,-0.788812,0.217639,-0.333333
4,FL,-0.333333,-0.555556,0.070707,-0.954023,NaN,0.400000,0.052632,-0.376623,-0.694915,-0.333333,-0.692016,-0.898127,0.129445,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,AK,NaN,NaN,-0.777778,-0.701149,NaN,-0.833333,-0.473684,-0.714286,-1.000000,-1.000000,-0.907529,-0.906082,0.041252,1.000000
95408,TX,1.000000,1.000000,-0.616162,-1.000000,NaN,-0.900000,-0.684211,-0.818182,-1.000000,-1.000000,-0.914856,-0.913523,-0.041252,0.333333
95409,MI,NaN,NaN,-0.595960,-1.000000,NaN,-0.566667,-0.263158,-0.584416,-0.949153,0.333333,-0.914856,-0.804465,-0.650071,-0.333333
95410,CA,1.000000,NaN,-0.595960,-0.977011,0.555556,0.166667,-0.052632,-0.220779,-0.661017,1.000000,-0.422941,-0.952784,-0.911807,0.333333


## Missing Values

In [37]:
df_MV = df_FEreduction.copy()

In [38]:
df_numeric, df_categorical = splitNumericAndCategorical(df_MV)

Total Columns: 15
Numerical Columns: 14
Categorical Columns: 1


In [39]:
imputer = KNNImputer(n_neighbors = 5, weights = 'uniform')
df_MV[df_numeric.columns] = pd.DataFrame(imputer.fit_transform(df_numeric))

## Outlier Removal

In [40]:
df_outlier = df_MV.copy()

In [41]:
df_numeric, df_categorical = splitNumericAndCategorical(df_outlier)

Total Columns: 15
Numerical Columns: 14
Categorical Columns: 1


In [42]:
from sklearn.ensemble import IsolationForest

IsoForest = IsolationForest(contamination=0.1)

Labeling = IsoForest.fit_predict(df_numeric)

Labeling_bool = [True if x == 1 else False for x in Labeling]
Outlier_Labeling_bool = [False if x == 1 else True for x in Labeling]

df_NonOutlier = df_outlier.loc[Labeling_bool]
df_Outlier = df_outlier.loc[Outlier_Labeling_bool]

# Save Results

In [47]:
df_NonOutlier.to_csv('./Outputs/dataprep_NonOutliers.csv',index=False)
df_Outlier.to_csv('./Outputs/dataprep_Outliers.csv',index=False)
df_Prepped.to_csv('./Outputs/dataprep_final.csv',index=False)